In [2]:
#reomve when pushing to production
import findspark
findspark.init(spark_home="/home/hdoop/spark-3.2.2-bin-hadoop3.2")

from pyspark.sql import functions as f, SparkSession,Row
import json,os
import path_clm as pc

from pyspark.sql.types import *

In [3]:
path = pc.Path()

def config():
    with open(path.config) as f:
        return json.loads(f.read())

config = config()

In [5]:
spark = (SparkSession
        .builder
        .appName("CaseStudy - MovieLenz")
        .master("local[6]").enableHiveSupport()
        .getOrCreate())

# spark.sparkContext.getConf().getAll()

2022-10-15 16:45:11,490 WARN util.Utils: Your hostname, shiva-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2022-10-15 16:45:11,492 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
SourceMacroDict = {}

SourceMacroDict['path'] = config['paths']['source_dir']

##creating daa frames
df = {}

# df['genome-scores'] = spark.read.csv(f"{SourceMacroDict['path']}/genome-scores.csv",header=True,schema=StructType([StructField("movieID",IntegerType(),True),StructField("tagID",StringType(),True),StructField("relevance",DoubleType()) ]))

df['genome-tags'] = spark.read.csv(f"{SourceMacroDict['path']}/genome-tags.csv",header=True,schema=StructType([StructField("tagID",IntegerType()),StructField("tag",StringType())]))

df['links'] = spark.read.csv(f"{SourceMacroDict['path']}/links.csv",header=True
                             ,schema=StructType([StructField("movieId", IntegerType()), StructField("imdbId", IntegerType()), StructField("tmdbId", IntegerType())]))

df['movies'] = spark.read.csv(f"{SourceMacroDict['path']}/movies.csv",header=True,
                              schema=StructType([StructField("movieId", IntegerType()), StructField("title",StringType()), StructField("genres",StringType())]))

df['ratings'] = spark.read.csv(f"{SourceMacroDict['path']}/ratings.csv",header=True,
                               schema=StructType([StructField("userId",IntegerType()),StructField("movieId",IntegerType()),StructField("rating",DoubleType()),StructField("timestamp",StringType())]))

df['tags'] = spark.read.csv(f"{SourceMacroDict['path']}/tags.csv",header=True,schema=StructType([StructField("userId",IntegerType())
                                                                                                 ,StructField("movieId",IntegerType()),StructField("tag",StringType()),StructField("timestamp",StringType())]))

df['ratings'] = df['ratings'].withColumn("ratings_timestamp", f.expr("to_timestamp(from_unixtime(timestamp))"))

df['tags'] = df['tags'].withColumn("tags_timestamp", f.expr("to_timestamp(from_unixtime(timestamp))"))

In [7]:
df['movies'] = df['movies'].withColumn("releasedInyear",f.expr("cast(substr(split(title,'[(]')[1],1,4) as Integer)"))

In [8]:
#final report 
df['final_report'] = (
    spark.table('genome_scores').where("relevance > 0.5").join(df['genome-tags'],['tagid'],'inner')
    .join(
        df['movies'].where("releasedInyear is not null") ,['movieID'],'inner'
    )
    .join(
        df['ratings'].groupBy("movieId").agg(f.round(f.avg("rating"),2).alias("total_ratings")),['movieId'],'inner'
    )
)

2022-10-15 16:45:30,743 WARN conf.HiveConf: HiveConf of name hive.metastore.event.db.notification.api.auth does not exist
2022-10-15 16:45:30,743 WARN conf.HiveConf: HiveConf of name hive.server2.active.passive.ha.enable does not exist
2022-10-15 16:45:30,744 WARN conf.HiveConf: HiveConf of name hive.hwi.listen.host does not exist
2022-10-15 16:45:30,744 WARN conf.HiveConf: HiveConf of name hive.server2.thrift.bind.port does not exist
2022-10-15 16:45:30,745 WARN conf.HiveConf: HiveConf of name hive.hwi.listen.port does not exist


In [20]:
df['final_report'].write.parquet("s3a://movielenzreports/casestudy/movielenz_final",mode="overwrite")

In [9]:
df['final_report'] = spark.read.parquet("s3a://movielenzreports/casestudy/movielenz_final")

In [10]:
# query to report movie name year adn total ratings along with geners
df['movie_report'] = (
    df['final_report'].groupBy("movieId","title").agg(f.round(f.avg("total_ratings"),2).alias("total_ratings"),
    f.first("releasedinyear").alias("releasedinyear"),f.first("genres").alias("genres") )
    .withColumn("total_genres_covered", f.expr("size(split(genres,'[|]'))"))
)

In [11]:
# config['mysql']

In [15]:
(
    df['movie_report']
    .write.format("jdbc")
    .option("url",config['mysql']['url'])
    .option("user",config['mysql']['user'])
    .option("password",config['mysql']['password'])
    .option("batchSize",10000)
    .option("dbtable","movie_report")
    .option("driver","com.mysql.cj.jdbc.Driver")
    .save()
)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
